# Writing an efficient code for GeoPandas and Shapely in 2023

With the release of Shapely 2.0, the GeoPandas-based code that have been optimised years ago may no longer provide the best performance. This workshop will show you how to change that and write efficient and convenient GeoPandas code that uses the benefits of the latest developments in the Python geospatial ecosystem.

**Martin Fleischmann, Joris van den Bossche**

08/03/2022, Basel

## Setup

Follow the ReadMe to set up the environment correctly. You should have these packages installed:

```
- geopandas
- pyogrio
- pyarrow
```

## What is GeoPandas?

**Easy, fast and scalable geospatial analysis in Python**

From the docs:

> The goal of GeoPandas is to make working with geospatial data in python easier. It combines the capabilities of pandas and shapely, providing geospatial operations in pandas and a high-level interface to multiple geometries to shapely. GeoPandas enables you to easily do operations in python that would otherwise require a spatial database such as PostGIS.

## How to write an efficient code for GeoPandas?

The way an efficient code should look like evolves as the whole ecosystem constantly develops better, smarter and faster tools. What was considered a good piece of code only a few years ago may not be optimal today. 

This notebook contains a set of examples of the common tasks. Each shows a way that was recommended some time ago. By us, by the community, in the documentation or on StackOverflow or elsewhere. And each show a way that is recommended today, with GeoPandas 0.12 and Shapely 2.0. 

In [1]:
import geopandas
import numpy
import pandas
import shapely

from shapely.geometry import Point, LineString, Polygon

Check that you are running the latest versions. GeoPandas should be at 0.12.2 while Shapely at 2.0.1. 

In [2]:
geopandas.__version__, shapely.__version__

('0.12.2', '2.0.1')

- [ ] spatial predicates - use STRtree not binary predicates or sjoin
    - https://stackoverflow.com/questions/48097742/geopandas-point-in-polygon
    - https://stackoverflow.com/questions/62410871/how-do-i-test-if-point-is-in-polygon-multipolygon-with-geopandas-in-python
    - https://gis.stackexchange.com/questions/281652/finding-all-neighbors-using-geopandas
- [ ] efficient IO
    - pyogrio, parquet
- [ ] pairwise distance (shapely ufunc broadcasting - `shapely.distance(pts, np.reshape(pts, (-1, 1)))`)
    - https://stackoverflow.com/questions/64754025/calculate-all-distances-between-two-geodataframe-of-points-in-geopandas
- [x] folium mapping
    - https://autogis-site.readthedocs.io/en/latest/lessons/lesson-5/interactive-maps.html
- [x] WKT/WKB serialization
    - https://stackoverflow.com/questions/61122875/geopandas-how-to-read-a-csv-and-convert-to-a-geopandas-dataframe-with-polygons
    - https://stackoverflow.com/questions/61125808/geopandas-how-to-convert-the-column-geometry-to-string
- [x] creating point geometry
    - https://stackoverflow.com/questions/50971914/what-is-the-most-efficient-way-to-convert-numpy-arrays-to-shapely-points
- [ ] nearest
    - https://stackoverflow.com/questions/30740046/calculate-distance-to-nearest-feature-with-geopandas
    - https://stackoverflow.com/questions/56520780/how-to-use-geopanda-or-shapely-to-find-nearest-point-in-same-geodataframe
- [x] points to lines
    - https://stackoverflow.com/questions/51071365/convert-points-to-lines-geopandas
- [x] specifying a projection (don't use `init...`)
    - https://gis.stackexchange.com/questions/218450/getting-polygon-areas-using-geopandas
- [x] getting coordinates
    - https://gis.stackexchange.com/questions/287306/listing-all-polygon-vertices-coordinates-using-geopandas
- [x] dissolve connected components
    - https://gis.stackexchange.com/questions/271733/geopandas-dissolve-overlapping-polygons
- [x] rounding coordinates (and the issues it may cause)
    - https://gis.stackexchange.com/questions/321518/rounding-coordinates-to-five-decimals-in-geopandas

## Data creation

Before you can work with geometries, you may need to create them.

### Create points from coordinates

> **QUESTION:** I have an array with latitude and longitude coordinates denoting locations of interest. How do I create geometry?

Create a dummy array of coordinates, representing latitude and longitude.

In [ ]:
latitude = numpy.random.uniform(-90, 90, 100_000)
longitude = numpy.random.uniform(-180, 180, 100_000)

In [ ]:
coordinates_df = pandas.DataFrame({"latitude": latitude, "longitude": longitude})
coordinates_df.head()

#### The code of the past

Using `map` and a scalar `Point` object.

In [ ]:
# do not use this code

%time s = geopandas.GeoSeries(map(Point, zip(longitude, latitude)), crs=4326)
s.head()

Using list comprehension and a scalar `Point` object.

In [ ]:
# do not use this code

%time s = geopandas.GeoSeries([Point(coords) for coords in zip(longitude, latitude)], crs=4326)
s.head()

Using `apply` and a scalar `Point` object.

In [ ]:
# do not use this code

%time s = coordinates_df.apply(lambda x: Point(x[::-1]), axis=1)
s.head()

The resulting object is not even a GeoSeries yet.

In [ ]:
s = geopandas.GeoSeries(s, crs=4326)
s.head()

#### The code of today

Using `GeoSeries.from_xy`.

In [ ]:
%time s = geopandas.GeoSeries.from_xy(longitude, latitude, crs=4326)
s.head()

Using `geopandas.points_from_xy`.

In [ ]:
%time ga = geopandas.points_from_xy(coordinates_df.longitude, coordinates_df.latitude, crs=4326)
ga

### Geometry from text

> **QUESTION** I have loaded a CSV, like the one in `data/sales.csv`, that has a column looking like a geometry but it is a string. How do I create a geometry I can work with?

Load the CSV in question.

In [ ]:
df = pandas.read_csv("data/sales.csv")
df.head()

In [ ]:
df.dtypes

#### The code of the past

Using `shapely.wkt.loads`.

In [ ]:
# do not use this code
from shapely import wkt

%time gdf = geopandas.GeoDataFrame(df, geometry=df.geometry.apply(wkt.loads), crs=4326)
gdf.head()

#### The code of today

Using `GeoSeries.from_wkt`.

In [ ]:
%time gdf = geopandas.GeoDataFrame(df, geometry=geopandas.GeoSeries.from_wkt(df.geometry, crs=4326))
gdf.head()

In [ ]:
gdf.dtypes

### Geometry from bytes

> **QUESTION** I have loaded a Apache Parquet, like the one in `data/sales.parquet`, that has a column called geometry but it looks weird. How do I create a geometry I can work with?

Load the CSV in question.

In [ ]:
df = pandas.read_parquet("data/sales.parquet")
df.head()

#### The code of the past

Using `shapely.wkb.loads`.

In [ ]:
# do not use this code
from shapely import wkb

%time gdf = geopandas.GeoDataFrame(df, geometry=df.geometry.apply(wkb.loads), crs=4326)
gdf.head()

#### The code of today

Using `GeoSeries.from_wkb`.

In [ ]:
%time gdf = geopandas.GeoDataFrame(df, geometry=geopandas.GeoSeries.from_wkb(df.geometry, crs=4326))
gdf.head()

In [ ]:
gdf.dtypes

#### Side note

The same would apply if you wanted to dump geometry to WKT/WKB. Do not use `wkt.dumps` or `wkb.dumps` on scalar geometries. Use `to_wkt` or `to_wkb` instead.

In [ ]:
# do not use this code
gdf.geometry.apply(wkb.dumps)

In [ ]:
gdf.to_wkt()

In [ ]:
gdf.to_wkb()

### Lines from points

> **QUESTION:** I have coordinates representing trajectories. How do I create LineString geometries from those?

In [ ]:
df = pandas.DataFrame(
    {"lat": latitude, "lon": longitude, "traj_id": numpy.repeat(numpy.arange(12500), 8)}
)
df

#### The code of the past

Using groupby-apply with a scalar `LineString` constructor after creation of `Point` geometry.

In [ ]:
%%time
# do not use this code
grouped = df.groupby(["traj_id"])[["lon", "lat"]].apply(lambda x: LineString(x))
grouped = geopandas.GeoDataFrame(grouped.to_frame("geometry"), crs=4326)
grouped

The same with unnecessary creating of Point geometry (a very common pattern and a top answer on StackOverflow).

In [ ]:
%%time
# do not use this code
geometry = [Point(xy) for xy in zip(df["lon"], df["lat"])]
gdf = geopandas.GeoDataFrame(df.copy(), geometry=geometry)

grouped = gdf.groupby("traj_id")["geometry"].apply(lambda x: LineString(x.tolist()))
grouped = geopandas.GeoDataFrame(grouped, crs=4326)
grouped

#### The code of today

Use `shapely.linestrings` constructor.

In [ ]:
%%time
gdf = geopandas.GeoDataFrame(
    geometry=shapely.linestrings(df[["lon", "lat"]], indices=df.traj_id),
)
gdf

What if the trajectory index is not `0...N`?

In [ ]:
df["traj_id"] = df["traj_id"] * 3
df

Use `pandas.factorize` to enumerate unique values. But remember - the DataFrame should have all points belonging to a same trajectory together. Try sorting first if that is not the case.

In [ ]:
%%time
gdf = geopandas.GeoDataFrame(
    geometry=shapely.linestrings(
        df[["lon", "lat"]], indices=pandas.factorize(df.traj_id)[0]
    ),
    crs=4326,
)
gdf

What does `pandas.factorize` do?

In [ ]:
pandas.factorize(df.traj_id)

You can use the other array as an index.

In [ ]:
%%time
indices, index = pandas.factorize(df.traj_id)
gdf = geopandas.GeoDataFrame(
    geometry=shapely.linestrings(df[["lon", "lat"]], indices=indices),
    index=index,
    crs=4326,
)
gdf

## Coordinate operations

Sometimes you need to work with individual coordinates, not geometries.

### Get coordinates as an array

> **QUESTION:** How to list all coordinates of all vertices of LineStrings in my GeoDataFrame?

In [ ]:
gdf.head()

#### The code of the past

Looping through geometries and extracting coordinates via `.coords.xy`, then concatenating all together.

In [ ]:
%%time
# do not use this code
coordinates = numpy.hstack([geom.coords.xy for geom in gdf.geometry]).T
coordinates

#### The code of today

Use `shapely.get_coordinates` directly on a GeoSeries.

In [ ]:
%%time
coordinates = shapely.get_coordinates(gdf.geometry)
coordinates

You can even get an index.

In [ ]:
%%time
coordinates, index = shapely.get_coordinates(gdf.geometry, return_index=True)
index

#### The code of the future (sneak-peek of upcoming GeoPandas 0.13)

Use `GeoSeries.get_coordinates()`.

In [ ]:
%%time
gdf.get_coordinates(index_parts=True)

### Rounding coordinates

> **QUESTION:** I would like to round the coordinates to 3 decimals. How?

In [ ]:
gdf.head()

#### The code of the past

Using formatting of a WKT representation and regex.

In [ ]:
# do not use this code
import re
from shapely.wkt import loads


def mround(match):
    return "{:.3f}".format(float(match.group()))


simpledec = re.compile(r"\d*\.\d+")

%time rounded = gdf.geometry.apply(lambda x: loads(re.sub(simpledec, mround, x.wkt)))
rounded.head()

#### The code of today

Use `shapely.set_precision` on a GeoSeries.

In [ ]:
%time shapely.set_precision(gdf.geometry, 0.001)

As a bonus, you can specify how to handle geometries that became invalid due to the rounding. See the [documentation](https://shapely.readthedocs.io/en/stable/reference/shapely.set_precision.html?highlight=precision#shapely.set_precision).

## Specify coordinate reference system (CRS)

> **QUESTION:** How to specify a CRS in `to_crs` or `set_crs`?

In [ ]:
gdf.crs

#### The code of the past

Using the dictionary with an `init` key.

In [ ]:
gdf.to_crs({'init': 'epsg:3857'})

### The code of today

Use anything accepted by [`pyproj.CRS.from_user_input()`](https://pyproj4.github.io/pyproj/stable/api/crs/crs.html#pyproj.crs.CRS.from_user_input) or an `epsg` keyword argument.

- PROJ string
- Dictionary of PROJ parameters
- PROJ keyword arguments for parameters
- JSON string with PROJ parameters
- CRS WKT string
- An authority string [i.e. 'epsg:4326']
- An EPSG integer code [i.e. 4326]
- A tuple of ("auth_name": "auth_code") [i.e ('epsg', '4326')]
- An object with a `to_wkt` method.
- A `pyproj.crs.CRS` class

In [ ]:
gdf.to_crs('epsg:3857')
# or
gdf.to_crs(3857)
# or
gdf.to_crs(epsg=3857)
# or 
gdf.to_crs(('epsg', '4326'))

## Interactive maps

> **QUESTION:** How to create an interactive map from my GeoDataFrame?

You can use a population grid from the great [AutoGIS course](https://autogis-site.readthedocs.io/en/latest/lessons/lesson-5/interactive-maps.html#add-a-polygon-layer).

In [ ]:
population_grid = (
    geopandas.read_file(
        "https://kartta.hsy.fi/geoserver/wfs"
        "?service=wfs"
        "&version=2.0.0"
        "&request=GetFeature"
        "&typeName=asuminen_ja_maankaytto:Vaestotietoruudukko_2020"
        "&srsName=EPSG:4326"
        "&bbox=24.6,60.1,25.2,60.4,EPSG:4326"
    )
    .set_crs("EPSG:4326")
)
population_grid.head()

### The code of the past

Use `folium` to construct the interactive map.

In [ ]:
# you can use this code, but it may be unnecessarily inconvenient
import folium

# define string ID
population_grid["id"] = population_grid.index.astype(str)

# initialise map. You need to know the location and a zoom level upfront
interactive_map = folium.Map(
    location=(60.17, 24.94),
    zoom_start=12
)

# create choropleth layer
population_grid_layer = folium.Choropleth(
    geo_data=population_grid,
    data=population_grid,
    columns=("id", "asukkaita"),
    key_on="feature.id",
    bins=9,
    fill_color="YlOrRd",
    line_weight=0,
    highlight=True
)

# add it to the map
population_grid_layer.add_to(interactive_map)


# define style function to hide the layer so we can use Tooltip over Choropleth
def style_function(feature):
    return {
        "color": "transparent",
        "fillColor": "transparent"
    }

# define tooltip
tooltip = folium.features.GeoJsonTooltip(
    fields=("asukkaita",),
)

# define layer that can host tooltip
tooltip_layer = folium.features.GeoJson(
    population_grid,
    style_function=style_function,
    tooltip=tooltip
)

# add the layer with a tooltip to the map
tooltip_layer.add_to(interactive_map)

interactive_map

### The code of today

Use built-in `.explore()` method. It wraps `folium` under the hood but makes the whole process much easier.  

In [ ]:
population_grid.explore("asukkaita", cmap="YlOrRd", tooltip='asukkaita', style_kwds=dict(stroke=False))

## Distance operations

In [3]:
homes = geopandas.read_file("data/sales.gpkg", engine="pyogrio").to_crs("EPSG:2926")
food = geopandas.read_file("data/food_establishments.gpkg", engine="pyogrio").to_crs("EPSG:2926")

In [4]:
print(len(homes), len(food))

21613 9595


### Pairwise distances

> **QUESTION:** How to calculate the distance between all points of two GeoDataFrames?

In [5]:
%%time
homes[:1000].geometry.apply(lambda geom: food.distance(geom))

CPU times: user 4.24 s, sys: 64.5 ms, total: 4.3 s
Wall time: 4.3 s


,0,1,2,3,4,5,6,7,8,9,...,9585,9586,9587,9588,9589,9590,9591,9592,9593,9594
0,55971.619780,53509.492302,37679.166846,46768.762723,40144.710785,71048.298626,16675.542689,27616.595831,90104.480202,78976.073754,...,24416.712655,65264.628021,37524.190200,61485.612070,60468.708740,74737.039431,60855.044758,91935.696200,75698.327734,27870.632741
1,22118.613577,26838.171046,43728.200955,33696.889898,42019.163043,47251.595528,93046.554985,105435.928071,24109.573303,6504.622744,...,78420.682907,29468.778605,41947.137640,31986.353230,19334.633500,152632.328070,17285.801480,32099.320511,2674.849656,105726.978096
2,31687.106167,41194.084960,55369.717464,46569.167111,54479.276750,30640.734341,95643.345995,108743.912706,7376.635624,28458.792097,...,89337.784587,19289.033686,52264.919454,23329.161089,35206.570662,156998.558453,27894.225330,11164.225298,21122.821654,109098.097047
3,56854.664642,48648.804266,33111.076058,42273.479051,34073.460974,88184.408247,47182.056613,52098.719787,94574.753187,73360.603552,...,10439.414031,75642.486388,36445.858505,72947.255352,55996.721491,88324.767872,61074.665858,99264.139052,73722.904121,52087.393194
4,64165.449259,72312.717552,70854.904880,70849.242392,72423.219018,33738.115775,66167.042267,77575.441353,70002.173744,82905.867346,...,86448.493212,48163.278873,66945.502050,45501.966081,73457.813744,121194.261370,66127.359921,65209.648754,74983.649046,78024.109377
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,131834.226587,132523.623022,118394.403897,126424.370458,120917.430306,125937.764693,64914.820939,57399.145298,157645.302207,155862.600257,...,99141.138073,131122.507958,117370.723527,127117.547877,138550.002516,50470.896883,136339.790281,155963.656390,150644.904784,57409.596322
996,140434.603574,136424.368263,119458.253261,129411.095672,121596.996819,151801.796360,72561.476342,59333.699554,174849.631981,162475.104312,...,87479.241536,149404.912330,120541.553463,145475.244255,143809.882096,12362.039430,145322.249675,176183.355136,160051.802746,58940.824476
997,8362.604435,8373.247073,25288.007771,15230.686382,23538.890483,48235.280390,76392.993959,88351.491897,38800.679551,18238.314457,...,59939.809637,29638.323464,23779.278759,29833.929325,1380.842190,134940.187066,7426.151666,45043.830812,16756.824837,88616.959581
998,120727.448282,121338.848742,107209.707031,115231.415549,109733.255195,115889.293805,53845.520787,46882.400163,147027.007431,144754.627850,...,88533.990169,120457.133618,106177.073212,116435.431345,127387.635527,47959.196755,125254.550429,145562.976141,139608.582460,46926.365476


In [6]:
%%time
shapely.distance(np.asarray(homes[:1000].geometry)[:, np.newaxis], np.asarray(food.geometry)[np.newaxis, :])

CPU times: user 4.38 s, sys: 8.84 ms, total: 4.39 s
Wall time: 4.39 s


array([[ 55971.61978011,  53509.49230159,  37679.16684576, ...,
         91935.69620004,  75698.32773388,  27870.63274051],
       [ 22118.61357747,  26838.17104637,  43728.2009546 , ...,
         32099.32051103,   2674.8496556 , 105726.97809612],
       [ 31687.10616708,  41194.08496025,  55369.71746413, ...,
         11164.2252977 ,  21122.82165427, 109098.09704707],
       ...,
       [  8362.60443455,   8373.24707282,  25288.00777085, ...,
         45043.83081178,  16756.82483665,  88616.95958064],
       [120727.44828204, 121338.84874241, 107209.70703059, ...,
        145562.97614113, 139608.58245991,  46926.36547589],
       [124086.40378131, 119768.97649102, 102755.22830292, ...,
        160725.28758407, 143626.23008534,  44422.74506001]])

### Distance to nearest point

> **QUESTION:** How to calculate the distance to the nearest feature of another GeoDataFrame?

#### The code of the past

Using a brute-force calculation of the distance for all points and taking the minimum:

In [7]:
%%time
# do not use this code
# only taking the first 1000 rows (1/20th of full dataset)
homes[:1000].geometry.apply(lambda geom: food.distance(geom).min())

CPU times: user 4.74 s, sys: 0 ns, total: 4.74 s
Wall time: 4.74 s


0      1381.123133
1       873.387816
2       591.259216
3       639.728786
4      1899.817107
          ...     
995    1404.505477
996     102.483623
997     949.162907
998    3309.531520
999    2022.061672
Name: geometry, Length: 1000, dtype: float64

Using the shapely's `nearest_point` to get the nearest point and only calculate the distance for this point:

In [8]:
from shapely.ops import nearest_points

In [9]:
%%time
# do not use this code

# unary union of the right geomtries (food establishment locations) 
points = food.geometry.unary_union

def nearest_dist(point, other=points):
    # find the nearest point and return the distance to that point
    nearest = nearest_points(point, other)[1]
    return point.distance(nearest)

homes[:1000].geometry.apply(lambda geom: nearest_dist(geom))

CPU times: user 995 ms, sys: 0 ns, total: 995 ms
Wall time: 996 ms


0      1381.123133
1       873.387816
2       591.259216
3       639.728786
4      1899.817107
          ...     
995    1404.505477
996     102.483623
997     949.162907
998    3309.531520
999    2022.061672
Name: geometry, Length: 1000, dtype: float64

#### The code of today

Use `sjoin_nearest` with returning the distance:

In [10]:
%%time
# this is now with the full dataset (size x20) !!
homes.sjoin_nearest(food[["geometry"]], how="left", distance_col="distance")

CPU times: user 393 ms, sys: 4.02 ms, total: 397 ms
Wall time: 399 ms


,price,bedrooms,geometry,index_right,distance
0,221900.0,3.0,POINT (1288600.213 189722.864),2976,1381.123133
1,538000.0,3.0,POINT (1274757.775 266526.003),1825,873.387816
2,180000.0,2.0,POINT (1296034.393 272292.738),5757,591.259216
3,604000.0,4.0,POINT (1255066.919 193875.327),2992,639.728786
4,510000.0,3.0,POINT (1341591.923 227340.234),1148,1899.817107
...,...,...,...,...,...
21608,360000.0,3.0,POINT (1267957.622 258741.556),8604,385.859981
21609,400000.0,4.0,POINT (1262652.024 190038.241),7563,1307.039875
21610,402101.0,2.0,POINT (1278800.266 220260.644),6316,866.514612
21611,400000.0,3.0,POINT (1335191.404 197418.062),4729,685.778669


If you only need the distances (and not joined dataframes), you can also use the spatial index directly:

In [11]:
%%time
tree = shapely.STRtree(food.geometry)
indices, distances = tree.query_nearest(homes.geometry, all_matches=False, return_distance=True)
distances

CPU times: user 360 ms, sys: 3.94 ms, total: 364 ms
Wall time: 362 ms


array([1381.12313323,  873.38781603,  591.25921553, ...,  866.51461204,
        685.77866919,  934.19186503])

### Distance to all points within a search radius

> **QUESTION:** How to calculate the distance between all points of two GeoDataFrames, but limited to distances below a certain threshold?

With many features, doing the full pairwise distance calculation is costly (and gives a large matrix as result). If you want more than just the nearest, another option is to limit it calculate the distance to points within a certain distance:

In [12]:
tree = shapely.STRtree(food.geometry)

In [13]:
%%time
idx1, idx2 = tree.query(homes.geometry, predicate="dwithin", distance=5_000)
distances = shapely.distance(np.asarray(homes.geometry)[idx1], np.asarray(food.geometry)[idx2])
distances

CPU times: user 1.33 s, sys: 7.16 ms, total: 1.34 s
Wall time: 1.34 s


array([4751.53915679, 3628.43746316, 1453.00709077, ..., 3839.70194129,
       3839.70591763, 3839.44665143])

In [14]:
len(distances)

1184314

## Merge overlapping or touching polygons (note: do spatial predicates first, this is just an application in principle)

**QUESTION:** How can I merge all polygons that overlap into one?

In [ ]:
gdf

In [ ]:
s = geopandas.GeoDataFrame(geometry=[Polygon([(0, 0), (0, 2), (2, 2), (2, 0)]), Polygon([(0, 1), (0, 3), (2, 3), (2, 1)]),Polygon([(1, 0), (1, 2), (3, 2), (3, 0)]), Polygon([(4, 4), (4, 6), (6, 6), (6, 4)])])
s.plot(cmap='tab10', alpha=.5)

### The code of the past

Create a connectivity matrix using spatial predicates like `overlaps` to get group overlapping polygons.

In [ ]:
from scipy.sparse.csgraph import connected_components

In [ ]:
%%timeit
# do not use this code
overlap_matrix = s.geometry.apply(lambda x: s.overlaps(x)).values.astype(int)
n, ids = connected_components(overlap_matrix)

In [ ]:
s.dissolve(ids).plot(cmap='tab10', alpha=.5)

The overlap matrix will have the shape `(N, N)`. That can be **a lot** of values, while most of them will be 0.

In [ ]:
overlap_matrix

### The code of today

Use spatial index query to create the connectivity matrix as a sparse array.

In [ ]:
from scipy.sparse import coo_array

In [ ]:
%%timeit
right_index, left_index = s.sindex.query_bulk(s.geometry, predicate="overlaps")
connectivity = coo_array(
        (
            numpy.ones(right_index.shape, dtype=bool),
            (right_index, left_index),
        ),
        shape=(s.shape[0], s.shape[0]),
        dtype=bool,
    )
n, ids = connected_components(connectivity)

In [ ]:
s.dissolve(ids).plot(cmap='tab10', alpha=.5)

This is would the connectivy looked if dense. But since it is sparse, neither of False is stored.

In [ ]:
connectivity.todense()

More convenient option would be to use external library from the geopandas ecosystem. This will create the sparse matrix for you.

In [ ]:
import libpysal

In [ ]:
connectivity = libpysal.weights.fuzzy_contiguity(s, predicate="overlaps")

In [ ]:
s.dissolve(connectivity.component_labels).plot(cmap='tab10', alpha=.5)

If you were interested in topological contguity, using `Queen` weights constructor will be even faster.

In [ ]:
connectivity = libpysal.weights.Queen.from_dataframe(s)